In [1]:
import pandas as pd
import geopandas as gpd
from geopandas.tools import geocodec

ImportError: cannot import name 'geocodec' from 'geopandas.tools' (C:\Users\Acer\AppData\Local\Programs\Python\Python312\Lib\site-packages\geopandas\tools\__init__.py)

In [ ]:

# Load the CSV file into a DataFrame
input_csv_path = "tree_loss_data.csv"  # Replace with the path to your CSV file
df = pd.read_csv(input_csv_path)

# Geocode the country names
geocoded = geocode(df["Country"], provider="nominatim", user_agent="geoapi")

# Extract latitude and longitude
geocoded["Latitude"] = geocoded.geometry.y
geocoded["Longitude"] = geocoded.geometry.x

# Merge geocoded data with the original DataFrame
df = pd.concat([df, geocoded[["Latitude", "Longitude"]]], axis=1)

# Convert to GeoDataFrame
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df["Longitude"], df["Latitude"])
)

# Save the updated GeoDataFrame to a GeoJSON or CSV file
output_path_geojson = "geocoded_tree_loss_with_weather.geojson"
output_path_csv = "geocoded_tree_loss_with_weather.csv"
gdf.to_file(output_path_geojson, driver="GeoJSON")
gdf.to_csv(output_path_csv, index=False)

print(f"Geocoded data saved to {output_path_geojson} and {output_path_csv}")

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import HeatMap


In [ ]:
# Load geocoded CSV
data_path = "geocoded_tree_loss_with_weather.csv"
df = pd.read_csv(data_path)

# Check data
print(df.head())


In [ ]:
# Tree Cover Loss Trend by Country
plt.figure(figsize=(12, 6))
sns.barplot(data=df, x="Country", y="Tree Cover Loss (2001–2023)", palette="viridis")
plt.xticks(rotation=90)
plt.title("Tree Cover Loss (2001–2023) by Country", fontsize=16)
plt.ylabel("Tree Cover Loss (hectares)")
plt.xlabel("Country")
plt.tight_layout()
plt.show()

# CO₂ Emissions Trend by Country
plt.figure(figsize=(12, 6))
sns.barplot(data=df, x="Country", y="CO₂ Emissions from Deforestation (Gt)", palette="magma")
plt.xticks(rotation=90)
plt.title("CO₂ Emissions from Deforestation by Country", fontsize=16)
plt.ylabel("CO₂ Emissions (Gt)")
plt.xlabel("Country")
plt.tight_layout()
plt.show()


In [ ]:
# Initialize a folium map
map_center = [0, 0]  # Approximate center of the world
heat_map = folium.Map(location=map_center, zoom_start=2)

# Prepare heatmap data
heat_data = df[["Latitude", "Longitude", "Tree Cover Loss (2001–2023)"]].dropna()
heat_data = heat_data.values.tolist()

# Add heat map layer
HeatMap(heat_data, radius=15, max_zoom=13, blur=10, gradient={0.4: 'blue', 0.65: 'lime', 1: 'red'}).add_to(heat_map)

# Save and display the map
heat_map.save("tree_loss_heatmap.html")
print("Heat map saved as 'tree_loss_heatmap.html'. Open this file in a browser to view the map.")
